In [6]:
import xarray as xr
import cftime 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import logging
import cartopy.crs as ccrs
import metpy  
import calendar
import argparse
import glob
from pygam import GAM, LinearGAM, s, f, te
from sklearn.model_selection import train_test_split


In [3]:
merged_dataset_annual_loop = pd.read_csv('merged_dataset_annual_loop.csv')

In [4]:
##make dataset into array for modeling 
X = np.array(merged_dataset_annual_loop[['stand_age','tair','co2' , 'current_veg']])
y = np.array(merged_dataset_annual_loop[['NCE_weighted']])

In [7]:
## train/test split

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.40, random_state=42)


In [8]:
##generalized additive model with three non linear variables (stand age, temperature, co2) and one categorical (PFTs)

gam = LinearGAM(s(0) + s(1) + s(2) + f(3)).fit(X_train, y_train)

In [9]:
gam.summary() ##note there is a bug with p-values, R does not have this issue with the GAM package

LinearGAM                                                                                                 
=============================================== ==========================================================
Distribution:                        NormalDist Effective DoF:                                     47.8802
Link Function:                     IdentityLink Log Likelihood:                               -6787541.371
Number of Samples:                      1504551 AIC:                                         13575180.5025
                                                AICc:                                        13575180.5057
                                                GCV:                                               35.8198
                                                Scale:                                             35.8177
                                                Pseudo R-Squared:                                   0.2333
Feature Function                  Lam

/tmp/ipykernel_86727/3358381670.py:1: UserWarning: KNOWN BUG: p-values computed in this summary are likely much smaller than they should be. 
 
Please do not make inferences based on these values! 

Collaborate on a solution, and stay up to date at: 
github.com/dswah/pyGAM/issues/163 

  gam.summary()


In [10]:
y_pred = gam.predict(X_test) ##predict 


In [11]:
##sample output file
sample_output = pd.DataFrame(
data = {
'predicted_NCE': y_pred
,'stand_age':X_test[:,0]
,'tair':X_test[:,1]
,'co2':X_test[:,2]
, 'current_veg': X_test[:,3]
    
}
)

In [14]:
sample_output.describe()

,predicted_NCE,stand_age,tair,co2,current_veg
count,1.003034e+06,1.003034e+06,1.003034e+06,1.003034e+06,1.003034e+06
mean,-1.158265e+00,2.353311e+02,8.931715e+00,4.552822e+02,3.234205e+01
std,3.298081e+00,1.902138e+02,5.024773e+00,1.852354e+02,3.781339e+00
min,-1.875464e+01,0.000000e+00,-1.099167e+01,2.892600e+02,1.000000e+01
25%,-1.313236e+00,1.030000e+02,5.167153e+00,3.071100e+02,3.300000e+01
50%,-3.611801e-01,2.170000e+02,8.958468e+00,3.649600e+02,3.300000e+01
75%,4.401282e-01,3.450000e+02,1.240000e+01,5.730000e+02,3.300000e+01
max,1.031390e+01,3.000000e+03,2.509917e+01,9.152900e+02,3.300000e+01


In [15]:
sample_output.loc[0:100000].to_csv('sample_output_for_tem_emulator_response_curves.csv') ##output first 100k rows